# Feature Transformation with a Amazon SageMaker Processing Job and SparkML

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge.

Often, distributed data processing frameworks such as Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Spark in a managed SageMaker environment to run our processing workload.

![](img/prepare_dataset.png)

![](img/processing.jpg)


## Contents

1. Setup
1. Use Amazon SageMaker Processing Job to execute a Spark ML Transformation
1. Setup Input Data
1. Setup Output Data
1. Build a Spark container for running the processing job
1. Run the Processing Job using Amazon SageMaker
1. Inspect the Processed Output Dta

## Setup

Add the following policies to your SageMaker Execution Role:  
* `EC2ContainerRegistry`
* Permissions: `List`, `Read`, `Write` 
* Repository:  `amazon-reviews-spark-processor`

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [1]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Using Amazon SageMaker Processing Job to Run a Spark ML Transformation

Get the name of previous Scikit Processing Job to use to construct the input for our Spark ML Processing Job.

# Build a Spark Docker Image to Run the Processing Job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset processing.

Build the example Spark container.

In [2]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [3]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  3.023MB
Step 1/33 : FROM openjdk:8-jre-slim
8-jre-slim: Pulling from library/openjdk

d04f60ab: Pulling fs layer 
c5772968: Pulling fs layer 
c6da18fe: Pulling fs layer 
Digest: sha256:3335eba51599781eb10a7bbee4237b3bbf146faef747320fa0b3a8c403f1d61f
Status: Downloaded newer image for openjdk:8-jre-slim
 ---> 381b20190cf7
Step 2/33 : RUN apt-get update
 ---> Running in 74863bc6480a
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:3 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:4 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Get:5 http://security.debian.org/debian-security buster/updates/main amd64 Packages [185 kB]
Get:6 http://deb.debian.org/debian buster-updates/main amd64 Packages [7380 B]
Fetched 8336 kB in 1s (6951 kB/s)
Reading package lists...
Removing intermediate container 74863bc6480a
 ---> 39561e9

Get:27 http://deb.debian.org/debian buster/main amd64 python3-minimal amd64 3.7.3-1 [36.6 kB]
Get:28 http://deb.debian.org/debian buster/main amd64 libmpdec2 amd64 2.4.2-2 [87.2 kB]
Get:29 http://deb.debian.org/debian buster/main amd64 libpython3.7-stdlib amd64 3.7.3-2+deb10u1 [1734 kB]
Get:30 http://deb.debian.org/debian buster/main amd64 python3.7 amd64 3.7.3-2+deb10u1 [330 kB]
Get:31 http://deb.debian.org/debian buster/main amd64 libpython3-stdlib amd64 3.7.3-1 [20.0 kB]
Get:32 http://deb.debian.org/debian buster/main amd64 python3 amd64 3.7.3-1 [61.5 kB]
Get:33 http://deb.debian.org/debian buster/main amd64 netbase all 5.6 [19.4 kB]
Get:34 http://deb.debian.org/debian buster/main amd64 bzip2 amd64 1.0.6-9.2~deb10u1 [48.4 kB]
Get:35 http://deb.debian.org/debian buster/main amd64 libapparmor1 amd64 2.13.2-10 [94.7 kB]
Get:36 http://deb.debian.org/debian buster/main amd64 libdbus-1-3 amd64 1.12.16-1 [214 kB]
Get:37 http://deb.debian.org/debian buster/main amd64 dbus amd64 1.12.16-1 [2

Get:120 http://deb.debian.org/debian buster/main amd64 libpython2-dev amd64 2.7.16-1 [20.9 kB]
Get:121 http://deb.debian.org/debian buster/main amd64 libpython-dev amd64 2.7.16-1 [20.9 kB]
Get:122 http://deb.debian.org/debian buster/main amd64 libpython3.7 amd64 3.7.3-2+deb10u1 [1498 kB]
Get:123 http://deb.debian.org/debian buster/main amd64 libpython3.7-dev amd64 3.7.3-2+deb10u1 [48.4 MB]
Get:124 http://deb.debian.org/debian buster/main amd64 libpython3-dev amd64 3.7.3-1 [20.1 kB]
Get:125 http://deb.debian.org/debian buster/main amd64 libsasl2-modules amd64 2.1.27+dfsg-1+deb10u1 [104 kB]
Get:126 http://deb.debian.org/debian buster/main amd64 libxml2 amd64 2.9.4+dfsg1-7+b3 [687 kB]
Get:127 http://deb.debian.org/debian buster/main amd64 manpages-dev all 4.16-2 [2232 kB]
Get:128 http://deb.debian.org/debian buster/main amd64 publicsuffix all 20190415.1030-1 [116 kB]
Get:129 http://deb.debian.org/debian buster/main amd64 python2.7-dev amd64 2.7.16-2+deb10u1 [294 kB]
Get:130 http://deb.deb

Selecting previously unselected package libpython3.7-stdlib:amd64.
Preparing to unpack .../libpython3.7-stdlib_3.7.3-2+deb10u1_amd64.deb ...
Unpacking libpython3.7-stdlib:amd64 (3.7.3-2+deb10u1) ...
Selecting previously unselected package python3.7.
Preparing to unpack .../python3.7_3.7.3-2+deb10u1_amd64.deb ...
Unpacking python3.7 (3.7.3-2+deb10u1) ...
Selecting previously unselected package libpython3-stdlib:amd64.
Preparing to unpack .../libpython3-stdlib_3.7.3-1_amd64.deb ...
Unpacking libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up python3-minimal (3.7.3-1) ...
Selecting previously unselected package python3.
(Reading database ... 10374 files and directories currently installed.)
Preparing to unpack .../000-python3_3.7.3-1_amd64.deb ...
Unpacking python3 (3.7.3-1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../001-netbase_5.6_all.deb ...
Unpacking netbase (5.6) ...
Selecting previously unselected package bzip2.
Preparing to unpack .../002-bzip2_1.0.6

Selecting previously unselected package libsasl2-modules-db:amd64.
Preparing to unpack .../050-libsasl2-modules-db_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-modules-db:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libsasl2-2:amd64.
Preparing to unpack .../051-libsasl2-2_2.1.27+dfsg-1+deb10u1_amd64.deb ...
Unpacking libsasl2-2:amd64 (2.1.27+dfsg-1+deb10u1) ...
Selecting previously unselected package libldap-common.
Preparing to unpack .../052-libldap-common_2.4.47+dfsg-3+deb10u1_all.deb ...
Unpacking libldap-common (2.4.47+dfsg-3+deb10u1) ...
Selecting previously unselected package libldap-2.4-2:amd64.
Preparing to unpack .../053-libldap-2.4-2_2.4.47+dfsg-3+deb10u1_amd64.deb ...
Unpacking libldap-2.4-2:amd64 (2.4.47+dfsg-3+deb10u1) ...
Selecting previously unselected package libnghttp2-14:amd64.
Preparing to unpack .../054-libnghttp2-14_1.36.0-2+deb10u1_amd64.deb ...
Unpacking libnghttp2-14:amd64 (1.36.0-2+deb10u1) ...
Selecting previously unsele

Selecting previously unselected package manpages-dev.
Preparing to unpack .../098-manpages-dev_4.16-2_all.deb ...
Unpacking manpages-dev (4.16-2) ...
Selecting previously unselected package publicsuffix.
Preparing to unpack .../099-publicsuffix_20190415.1030-1_all.deb ...
Unpacking publicsuffix (20190415.1030-1) ...
Selecting previously unselected package python2.7-dev.
Preparing to unpack .../100-python2.7-dev_2.7.16-2+deb10u1_amd64.deb ...
Unpacking python2.7-dev (2.7.16-2+deb10u1) ...
Selecting previously unselected package python2-dev.
Preparing to unpack .../101-python2-dev_2.7.16-1_amd64.deb ...
Unpacking python2-dev (2.7.16-1) ...
Selecting previously unselected package python-dev.
Preparing to unpack .../102-python-dev_2.7.16-1_amd64.deb ...
Unpacking python-dev (2.7.16-1) ...
Selecting previously unselected package python-pip-whl.
Preparing to unpack .../103-python-pip-whl_18.1-5_all.deb ...
Unpacking python-pip-whl (18.1-5) ...
Selecting previously unselected package python-p

Setting up libsasl2-2:amd64 (2.1.27+dfsg-1+deb10u1) ...
Setting up libmpx2:amd64 (8.3.0-6) ...
Setting up libubsan1:amd64 (8.3.0-6) ...
Setting up libisl19:amd64 (0.20-2) ...
Setting up libgirepository-1.0-1:amd64 (1.58.3-2) ...
Setting up libssh2-1:amd64 (1.8.0-2.1) ...
Setting up netbase (5.6) ...
Setting up python-pip-whl (18.1-5) ...
Setting up libkrb5-3:amd64 (1.17-3) ...
Setting up libmpdec2:amd64 (2.4.2-2) ...
Setting up libbinutils:amd64 (2.31.1-16) ...
Setting up cpp-8 (8.3.0-6) ...
Setting up libc-dev-bin (2.28-10) ...
Setting up readline-common (7.0-5) ...
Setting up publicsuffix (20190415.1030-1) ...
Setting up libxml2:amd64 (2.9.4+dfsg1-7+b3) ...
Setting up libcc1-0:amd64 (8.3.0-6) ...
Setting up liblocale-gettext-perl (1.07-3+b4) ...
Setting up liblsan0:amd64 (8.3.0-6) ...
Setting up libitm1:amd64 (8.3.0-6) ...
Setting up libreadline7:amd64 (7.0-5) ...
Setting up libgdbm6:amd64 (1.18.1-4) ...
Setting up gnupg-utils (2.2.12-1+deb10u1) ...
Setting up binutils-x86-64-linux-g

 ---> Running in 7565d8cc09bc
Removing intermediate container 7565d8cc09bc
 ---> afe1315d5230
Step 19/33 : ENV PATH $PATH:${SPARK_HOME}/bin
 ---> Running in 707c0e509ef8
Removing intermediate container 707c0e509ef8
 ---> 02af863c2fb4
Step 20/33 : RUN curl -sL --retry 3   "https://archive.apache.org/dist/spark/spark-${SPARK_VERSION}/${SPARK_PACKAGE}.tgz"   | gunzip   | tar x -C /usr/  && mv /usr/$SPARK_PACKAGE $SPARK_HOME  && chown -R root:root $SPARK_HOME
 ---> Running in 00d121a3bab5
Removing intermediate container 00d121a3bab5
 ---> b58fb1a7dca8
Step 21/33 : ENV PYSPARK_PYTHON=/usr/bin/python3
 ---> Running in 710be734dca3
Removing intermediate container 710be734dca3
 ---> 07f7e762f838
Step 22/33 : ENV PATH="/usr/bin:/opt/program:${PATH}"
 ---> Running in e9af66849acd
Removing intermediate container e9af66849acd
 ---> d91cd043f8fa
Step 23/33 : ENV YARN_RESOURCEMANAGER_USER="root"
 ---> Running in 9bb80b80aa1f
Removing intermediate container 9bb80b80aa1f
 ---> 159dd9465205
Step 24/33 

Create an Amazon Elastic Container Registry (Amazon ECR) repository for the Spark container and push the image.

In [4]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor:latest


Create ECR repository and push docker image

In [5]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [6]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo

{
    "repositories": [
        {
            "repositoryArn": "arn:aws:ecr:us-east-1:835319576252:repository/amazon-reviews-spark-processor",
            "registryId": "835319576252",
            "repositoryName": "amazon-reviews-spark-processor",
            "repositoryUri": "835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor",
            "createdAt": 1581702437.0,
            "imageTagMutability": "MUTABLE",
            "imageScanningConfiguration": {
                "scanOnPush": false
            }
        }
    ]
}


In [7]:
!docker tag $docker_repo:$docker_tag $image_uri

In [8]:
!docker push $image_uri

The push refers to repository [835319576252.dkr.ecr.us-east-1.amazonaws.com/amazon-reviews-spark-processor]

dfa45a48: Preparing 
b7e6cea0: Preparing 
1da39660: Preparing 
8ba09a76: Preparing 
85efb735: Preparing 
cc4d9d18: Preparing 
bf0edb94: Preparing 
934b8dc8: Preparing 
7e0592ba: Preparing 
c67dae8f: Preparing 
873d5643: Preparing 
f99cd11c: Preparing 
36ed7861: Preparing 
8dafa5c7: Preparing 
c67dae8f: Pushed   490.3MB/481.8MBPushing  11.96MB/191MBlatest: digest: sha256:c01bc01bf3c1956309691236b6ac1b6495b58721f86f5895ace19e2432980c8d size: 3472


# Run the Job using Amazon SageMaker Processing Jobs

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a SparkML script for processing in the job configuration.

Review the Spark processing script.

In [9]:
cat preprocess-spark-text-to-tfidf.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, DateType
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType
from pyspark.ml.feature import PCA, StandardScaler


def to_array(col):
    def to_array_internal(v):
        if v:
            return v.toArray().tolist()
        else:
            print('EmptyV: {}'.format(v))
            return []
    return udf(to_array_internal, ArrayType(DoubleType())).asNondeterministic()(col)


def transform(spark, s3_input_data, s3_output_train_data, s3_output_validation_data, s3_output_tes

In [10]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.8xlarge',
                            env={'mode': 'python'})

# Start the Spark Processing Job

_Notes on Invoking from Lambda:_
* However, if we use the boto3 SDK (ie. with a Lambda), we need to copy the `preprocess-text-to-tfidf.py` file to S3 and specify the everything include --py-files, etc.
* We would need to do the following before invoking the Lambda:
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/code/preprocess.py
     !aws s3 cp preprocess.py s3://<location>/sagemaker/spark-preprocess-reviews-demo/py_files/preprocess.py
* Then reference the s3://<location> above in the --py-files, etc.
* See Lambda example code in this same project for more details.

_Notes on not using ProcessingInput and Output:_
* Since Spark natively reads/writes from/to S3 using s3a://, we can avoid the copy required by ProcessingInput and ProcessingOutput (FullyReplicated or ShardedByS3Key) and just specify the S3 input and output buckets/prefixes._"
* See https://github.com/awslabs/amazon-sagemaker-examples/issues/994 for issues related to using /opt/ml/processing/input/ and output/
* If we use ProcessingInput, the data will be copied to each node (which we don't want in this case since Spark already handles this)

# Setup Input Data

In [11]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


In [12]:
!aws s3 ls $s3_input_data

2020-03-22 21:56:11   94010685 amazon_reviews_us_Software_v1_00.tsv.gz


# Setup Output Data

In [13]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

In [14]:
train_data_tfidf_output = 's3://{}/{}/output/tfidf-train'.format(bucket, output_prefix)
validation_data_tfidf_output = 's3://{}/{}/output/tfidf-validation'.format(bucket, output_prefix)
test_data_tfidf_output = 's3://{}/{}/output/tfidf-test'.format(bucket, output_prefix)

print(train_data_tfidf_output)
print(validation_data_tfidf_output)
print(test_data_tfidf_output)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-30-04-12-49/output/tfidf-train
s3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-30-04-12-49/output/tfidf-validation
s3://sagemaker-us-east-1-835319576252/amazon-reviews-spark-processor-2020-03-30-04-12-49/output/tfidf-test


In [15]:
from sagemaker.processing import ProcessingOutput

processor.run(code='preprocess-spark-text-to-tfidf.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_train_data', train_data_tfidf_output,
                         's3_output_validation_data', validation_data_tfidf_output,
                         's3_output_test_data', test_data_tfidf_output,                         
              ],
              # We need this dummy output to allow us to call 
              #    ProcessingJob.from_processing_name() later 
              #    to describe the job and poll for Completed status
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='dummy-output',
                                        source='/opt/ml/processing/output')
              ],          
              logs=True,
              wait=False
)


Job Name:  spark-amazon-reviews-processor-2020-03-30-04-12-49-546
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-processor-2020-03-30-04-12-49-546/input/code/preprocess-spark-text-to-tfidf.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-processor-2020-03-30-04-12-49-546/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [16]:
from IPython.core.display import display, HTML

spark_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']

display(HTML('<b>Review <a href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, spark_processing_job_name)))


In [17]:
from IPython.core.display import display, HTML

# This is different than the job name because we are not using ProcessingOutput's in this Spark ML case.
spark_processing_job_s3_output_prefix = output_prefix

display(HTML('<b>Review <a href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Spark Job Has Completed</b>'.format(bucket, spark_processing_job_s3_output_prefix, region)))


# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [18]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=spark_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-processor-2020-03-30-04-12-49-546/input/code/preprocess-spark-text-to-tfidf.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'dummy-output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-835319576252/spark-amazon-reviews-processor-2020-03-30-04-12-49-546/output/dummy-output', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]}, 'ProcessingJobName': 'spark-amazon-reviews-processor-2020-03-30-04-12-49-546', 'ProcessingResources': {'ClusterConfig': {'InstanceCount': 2, 'InstanceType': 'ml.r5.8xlarge', 'VolumeSizeInGB': 30}}, 'StoppingCondition': {'MaxRuntimeInSeconds': 86400}, 'AppSpecification': {'ImageUri': '835319576252.dkr.ecr.us-east-1.amazonaws

# Inspect the Processed Output Dataset


## The next cells will not work properly until the job completes above.


Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [19]:
!aws s3 ls --recursive $train_data_tfidf_output/

In [20]:
!aws s3 ls --recursive $validation_data_tfidf_output/

In [21]:
!aws s3 ls --recursive $test_data_tfidf_output/

# Pass `spark_processing_job_s3_output_prefix` as input to the next notebook

In [22]:
print(spark_processing_job_s3_output_prefix)

amazon-reviews-spark-processor-2020-03-30-04-12-49


In [23]:
%store spark_processing_job_s3_output_prefix


Stored 'spark_processing_job_s3_output_prefix' (str)
